<h1>Part 1: Read & Clean Up</h1>

In [ ]:
#Import dependencies
import pandas as pd
import numpy as np
from nodejs import node, npm, npx

In [ ]:
#Path to database
warn_notice_path = '/Users/leslieleiva/Documents/GitHub/WARN-Notice-Analysis/Resources/July2023-April2024_Warn_Report.xlsx'

#Read and convert xlsx files into dataframe
warn_df = pd.DataFrame(pd.read_excel(warn_notice_path,
                                     sheet_name=2,
                                     skiprows=1))

#Drop NaN rows in DF
warn_df = warn_df.dropna()

#Delete unnecesary columns
del warn_df['Address']

#Rename Columns to remove '\n'
warn_df = warn_df.rename(columns={
    warn_df.columns[0] : 'County',
    warn_df.columns[1] : 'Notice Date',
    warn_df.columns[2] : 'Processed Date',
    warn_df.columns[3] : 'Effective Date',
    warn_df.columns[5] : 'Layoff Status',
    warn_df.columns[6] : 'No. of Employees'

})

#Remove 'County' from the values in 'County'
warn_df['County'] = warn_df['County'].str.replace('County','')

#Review df
warn_df.head()

In [ ]:
company_list = sorted(list(warn_df['Company'].unique()))

#Creating a dictionary for autocorrecting Company names
correction_dict = {
    '99 Cents Only Stores':'99 Cents Only Stores LLC',
    'A.C. Vroman, Inc.':"A.C. Vroman, Inc. dba Vroman's Bookstore and Book Soup",
    'ABM General Services':['ABM General Services, Inc. at FAT 1 Amazon',
                            'ABM General Services, Inc. at ONT 6 Amazon',
                            'ABM General Services, Inc. at PSP 1 Amazon',
                            'ABM General Services, Inc. at SBD 1 Amazon',
                            'ABM General Services, Inc. at SNA 4 Amazon'],
    'ACRT Pacific':'ACRT Pacific',
    'ADT Solar':'ADT Solar',
    'AHMC San Gabriel Valley Medical Center':'AHMC San Gabriel Valley Medical Center LP dba San Gabriel Valley Medical Center',
    'AMP Printing':['AMP Printing',
                    'AMP Printing - Mailing Address'],
    'AMQ Solutions':'AMQ Solutions, LLC',
    'ATI Restoration':'ATI Restoration, LLC',
    'AVMAC':'AVMAC LLC',
    'Abbott':['Abbott Laboratories',
              'Abbott Vascular'],
    'Accenture':'Accenture',
    'Accesso':'Accesso',
    'Ace Hotel Group':'Ace Hotel Group LLC at Ace Hotel Los Angeles',
    'Active Wellness':'Active Wellness LLC',
    'Activision Blizzard':'Activision Blizzard',
    'Acutus Medical':'Acutus Medical, Inc.',
    'Adidas America':'Adidas America, Inc.',
    'Adventist Health':['Adventist Health',
                        'Adventist Health Simi Valley'],
    'Age of Learning':'Age of Learning, Inc.',
    'Agilent Technologies':['Agilent Technologes',
                            'Agilent Technologies',
                            'Agilent Technologies, Inc.'],
    'Ajinomoto Foods North America':'Ajinomoto Foods North America, Inc.',
    'Alchemee':'Alchemee, LLC',
    'Allakos':'Allakos Inc.',
    'Alsco':'Alsco Inc.',
    'Alstom Mass Transit Corp.':'Alstom Mass Transit Corp.',
    'Alto Operations California':'Alto Operations California, LLC',
    'Alvarado Hospital':'Alvarado Hospital, LLC dba Alvarado Hospital Medical Center',
    'Amazon':[

    ],
    'Amentum at NAS':['Amentum at NAS Lemoore',
                      'Amentum at NAS North Island'],
    'Amgen':'Amgen',
    'Amware Fulfillment':'Amware Fulfillment LLC',
    'Amyris':'Amyris, Inc.',
    'Analog Devices':'Analog Devices, Inc.',
    'Anaplan':'Anaplan, Inc.',
    'Anchor Brewing Company':'Anchor Brewing Company, LLC',
    'AppFolio':'AppFolio, Inc.',
    'Apple':'Apple Inc.',
    'Arden Hills Country Club':'Arden Hills Country Club, Inc',
    'Astra Space Operations':'Astra Space Operations, LLC',
    'Atara Biotherapeutics':'Atara Biotherapeutics',
    'Aura Management':['Aura Management Co',
                       'Aura Management Co.'],
    'Aurora Solar Inc.':'Aurora Solar Inc.',
    'B. Braun Medical Inc.':'B. Braun Medical Inc.',
    'BILL Operations':'BILL Operations, LLC',
    'BMO Bank':'BMO Bank N.A. successor in interest to Bank of the West',
    'Balfour Beatty US Civils':'Balfour Beatty US Civils',
    'Bamboo Group Operations':'Bamboo Group Operations LLC dba Bamboo Sushi',
    'Bandier West Hollywood':'Bandier West Hollywood LLC',
    'Bank of the West':'Bank of the West',
    'Baxalta US':['Baxalta US, Inc.',
                  'Baxalta US, Inc. dba Takeda'],
    'Bay Area PL Services':'Bay Area PL Services dba BayPLS',
    'Beauty & Essex':'Beauty & Essex',
    'Bechtel Infrastructure & Power Corporation':'Bechtel Infrastructure & Power Corporation',
    'Becton, Dickinson and Company':['Becton, Dickinson and Company',
                                     'Becton, Dickinson, and Company',
                                     'Becton, Dickinson, and Company - Pacific'],
    'Berry Global, Inc.':'Berry Global, Inc.',
    'Beyond Hello CA':'Beyond Hello CA, LLC',
    'Bionano Genomics':'Bionano Genomics, Inc.',
    'BlackLine Systems':'BlackLine Systems, Inc.',
    'Block, Inc.':'Block, Inc.',
    'Bloom Energy Corporation':'Bloom Energy Corporation',
    'Blowfish':'Blowfish',
    'Blue Shield of California':'Blue Shield of California',
    'Boardriders Wholesale':'Boardriders Wholesale, LLC',
    'Boeing Company':'Boeing Company',
    'Boston Scientific Corporation':'Boston Scientific Corporation',
    'Bridge Staffing':'Bridge Staffing, LLC',
    'BrightDrop':'BrightDrop',
    'Bristol Farms Newfound Market':'Bristol Farms Newfound Market Irvine Grocery Store',
    'Bristol Myers Squibb':'Bristol Myers Squibb',
    'Broadcom':'Broadcom Inc.',
    'Brookfield Properties':'Brookfield Properties (USA II) LLC',
    'Burke Williams':'Burke Williams',
    'BuzzFeed Media Enterprises':'BuzzFeed Media Enterprises, Inc.',
    'C&E GP Specialists':'C&E GP Specialists, Inc.',
    'CSL Vifor':'CSL Vifor',
    'Cacique Foods':'Cacique Foods, LLC',
    'Pizza Hut': [
        'CalPac Pizza II LLC dba Pizza Hut',
        'CalPac Pizza II dba Pizza Hut',
        'CalPac Pizza dba Pizza Hut',
        'CalPac Pizza dba Pizza Hut - 027383',
        'CalPac Pizza dba Pizza Hut - 027384',
        'CalPac Pizza dba Pizza Hut - 027385',
        'CalPac Pizza dba Pizza Hut - 027386',
        'CalPac Pizza dba Pizza Hut - 027398',
        'CalPac Pizza dba Pizza Hut - 027400',
        'CalPac Pizza dba Pizza Hut - 027401',
        'CalPac Pizza dba Pizza Hut - 027403',
        'CalPac Pizza dba Pizza Hut - 027406',
        'CalPac Pizza dba Pizza Hut - 027408',
        'Southern PacPizza dba Pizza Hut - 029197',
        'Southern PacPizza dba Pizza Hut - 029198',
        'Southern PacPizza dba Pizza Hut - 029199',
        'Southern PacPizza dba Pizza Hut - 029200',
        'Southern PacPizza dba Pizza Hut - 029201',
        'Southern PacPizza dba Pizza Hut - 029202',
        'Southern PacPizza dba Pizza Hut - 029203',
        'Southern PacPizza dba Pizza Hut - 029205',
        'Southern PacPizza dba Pizza Hut - 029207',
        'Southern PacPizza dba Pizza Hut - 029208',
        'Southern PacPizza dba Pizza Hut - 029210',
        'Southern PacPizza dba Pizza Hut - 029211',
        'Southern PacPizza dba Pizza Hut - 029212',
        'Southern PacPizza dba Pizza Hut - 029213',
        'Southern PacPizza dba Pizza Hut - 029215',
        'Southern PacPizza dba Pizza Hut - 029220',
        'Southern PacPizza dba Pizza Hut - 029221',
        'Southern PacPizza dba Pizza Hut - 029222',
        'Southern PacPizza dba Pizza Hut - 029224',
        'Southern PacPizza dba Pizza Hut - 029225',
        'Southern PacPizza dba Pizza Hut - 029226',
        'Southern PacPizza dba Pizza Hut - 029228',
        'Southern PacPizza dba Pizza Hut - 029229',
        'Southern PacPizza dba Pizza Hut - 029230',
        'Southern PacPizza dba Pizza Hut - 029232',
        'PacPartners dba Pizza Hut',
        'PacPizza dba Pizza Hut',
        'PacPizza dba Pizza Hut - 011664',
        'PacPizza dba Pizza Hut - 011728',
        'PacPizza dba Pizza Hut - 011729',
        'PacPizza dba Pizza Hut - 011730',
        'PacPizza dba Pizza Hut - 011753',
        'PacPizza dba Pizza Hut - 011755',
        'PacPizza dba Pizza Hut - 011761',
        'PacPizza dba Pizza Hut - 014697',
        'PacPizza dba Pizza Hut - 032516',
        'Pacpizza dba Pizza Hut'],
    'California Institute of Technology':'California Institute of Technology',
    'California Resources Corporation':'California Resources Corporation',
    'CamelBak Products':'CamelBak Products, LLC',
    'Capital One Financial':'Capital One Financial',
    'Carbon Health':['Carbon Health',
                     'Carbon Health Medical Group'],
    'Carbon, Inc.':'Carbon, Inc.',
    'Cardinal Glass Industries':'Cardinal Glass Industries, Inc.',
    'Catalent, Inc.':'Catalent, Inc.',
    'Centerra Group':'Centerra Group, LLC',
    'Central Admixture Pharmacy Services':'Central Admixture Pharmacy Services, Inc.',
    'Cepheid':'Cepheid',
    'Certified Freight Logistics':'Certified Freight Logistics, Inc.',
    'ChargePoint':'ChargePoint, Inc.',
    'Charles River Laboratories':'Charles River Laboratories, Inc.',
    'Charles Schwab & Co.':'Charles Schwab & Co., Inc.',
    'Chart Inc.':'Chart Inc.',
    'Checkr':'Checkr, Inc.',
    'Chick-fil-A':'Chick-fil-A',
    'Circle C Ranch':'Circle C Ranch, Inc.',
    'Cisco Systems':'Cisco Systems, Inc.',
    'City National Bank':'City National Bank',
    'Clari':'Clari Inc.',
    'Codexis':'Codexis, Inc.',
    'Coherent Corp.':'Coherent Corp.',
    'Cole Haan':'Cole Haan',
    'Collins Pine Company':'Collins Pine Company',
    'Command Security Service':'Command Security Service LP at Google/Bayview',
    'Company 3 Method':['Company 3 Method Inc.',
                        'Company 3/Method Inc'],
    'Comprehensive Autism Center':'Comprehensive Autism Center',
    'Container Connection':'Container Connection',
    'ContextLogic Inc.':'ContextLogic Inc.',
    'Coronado Brewing Company':'Coronado Brewing Company, Inc.',
    'Corteva Agriscience':'Corteva Agriscience LLC',
    'Cotton On La Mirada Distribution Center':'Cotton On La Mirada Distribution Center',
    'Country Club Mortgage':['Country Club Mortgage',
                             'Country Club Mortgage - Center',
                             'Country Club Mortgage - Hall'],
    'Covanta Long Beach Renewable Energy Corp.':'Covanta Long Beach Renewable Energy Corp.',
    'Covanta Projects':'Covanta Projects, LLC',
    'Covenant Living Communities & Services':'Covenant Living Communities & Services (Covenant Living West) dba Brandel Manor and Cypress Assisted',
    'Cracker Barrel':'Cracker Barrel',
    'Cruise LLC':['Cruise LLC - CC',
                  'Cruise LLC - Headquarters'],
    'Crunchbase':'Crunchbase',
    'Crystal Geyser Water Company':'Crystal Geyser Water Company',
    'Cue Health':['Cue Health - Moda South',
                  'Cue Health Inc',
                  'Cue Health Inc.',
                  'Cue Health, Moda North',
                  'Cue Health, Moda North II',
                  'Cue Health, Moda West',
                  'Cue Health, Vista',
                  'Cue Health, Waples'],
    'Cummins Meritor':'Cummins Meritor',
    'Curio Employer':['Curio Employer LLC',
                      'Curio Employer LLC dba The Hotel del Coronado, Curio Collection by Hilton'],
    'Cygnus Home Service':'Cygnus Home Service, LLC dba Yelloh',
    'DHL Supply Chain':'DHL Supply Chain',
    'Deliver-It' : ['DI Overnite LLC/DI Express LLC dba Deliver-IT',
                    'DI Overnite LLC/DI Express LLC dba Deliver-It'],
    'DXS Logistics':'DXS Logistics LLC',
    'Dabico Airport Solutions':'Dabico Airport Solutions, Inc.',
    'Daifuku Services America Corporation':'Daifuku Services America Corporation',
    'Dairy Farmers of America':'Dairy Farmers of America, Inc.',
    'Daiso Distribution Center':'Daiso Distribution Center',
    "David's Bridal":['David&rsquo;s Bridal, LLC',
                      "David's Bridal, LLC",
                      'David&rsquo;s Bridal, LLC'],
    'DazPak Flexible Packaging':'DazPak Flexible Packaging',
    'DePuy Synthes Products':'DePuy Synthes Products, Inc.',
    'Del Friscos Grille':'Del Friscos Grille',
    'Allen Brothers West Coast': ['Del Monte Capitol Meat Company, LLC DBA Allen Brothers West Coast',
                                  'Del Monte Capitol Meat Company, LLC dba Allen Brothers West Coast'],
    'Delta Dental Plan of California':'Delta Dental Plan of California',
    'Z Gallerie': ['DirectBuy Home Improvement, Inc. dba Z Gallerie',
                   'DirectBuy Home Improvement, Inc. dba Z Gellerie'],
    'Discord':'Discord Inc.',
    'Distribution Alternatives':'Distribution Alternatives, Inc.',
    'Divvy Homes':'Divvy Homes Inc.',
    'DreamWorks Animation':'DreamWorks Animation LLC',
    "Dreyer's Grand Ice Cream":["Dreyer's Grand Ice Cream",
                                "Dryer's Grand Ice Cream"],
    'Dynaflex Prodcuts':'Dynaflex Prodcuts',
    'E. J. Lauren':'E. J. Lauren, LLC',
    'EchoPark Automotive Long Beach':'EchoPark Automotive Long Beach',
    'Eclipse Advantage':'Eclipse Advantage, LLC at Rite Aid',
    'El Dorado National (California)':'El Dorado National (California), Inc.',
    'Elevance Health':'Elevance Health, Inc.',
    'Epic Games':'Epic Games',
    'Ericsson Inc.':'Ericsson Inc.',
    'Exabeam, Inc.':'Exabeam, Inc.',
    'Round Table Pizza':['Excalibur Pizza LLC',
                       'Excalibur Pizza LLC dba Round Table Pizza'],
    'Exelixis':'Exelixis, Inc.',
    'Experian Information Solutions':'Experian Information Solutions, Inc.',
    'FM Restaurants HQ':['FM Restaurants HQ, LLC',
                         'FM Restaurants HQ, LLC dba El Torito Mexican Grill'],
    'FUJIFILM Irvine Scientific':'FUJIFILM Irvine Scientific, Inc.',
    'FabFitFun Fulfillment Center':'FabFitFun Fulfillment Center',
    'Faire Wholesale':'Faire Wholesale, Inc.',
    'Family YMCA':'Family YMCA of the Desert at Palm Desert Aquatic Center',
    'Farmers Group, Inc.':['Farmers Group, Inc.',
                           'Farmers Group, Inc. - 6301'],
    'Farmers Insurance Exchange':['Farmers Insurance Exchange',
                                  'Farmers Insurance Exchange - 6303'],
    'Fashion Institute of Design & Merchandising':['Fashion Institute of Design & Merchandising',
                                                   'Fashion Institute of Design & Merchandising FIDM'],
    'FedEx Supply Chain, Inc.':'FedEx Supply Chain, Inc.',
    'Federal Express Corporation':'Federal Express Corporation',
    'FibroGen, Inc.':'FibroGen, Inc.',
    'First Financial Credit Union':'First Financial Credit Union',
    'Fish Market Restaurants, Inc.':'Fish Market Restaurants, Inc.',
    'Flex LTD':'Flex LTD',
    'Flexport':'Flexport, Inc.',
    'ForgeRock':'ForgeRock, Inc.',
    'Airtable':'Formagrid Inc. dba Airtable',
    'Foundation for California Community Colleges':'Foundation for California Community Colleges',
    'Front Porch Communities and Services':'Front Porch Communities and Services',
    'Full Steam Staffing':'Full Steam Staffing',
    'G&C Staffing':'G&C Staffing (The Parent Company - TPCO)',
    'GE United Technologies': ['GE United Technologies II Inc.',
                               'GE United Technologies LLC'],
    'GXO Logistics Supply Chain, Inc.':'GXO Logistics Supply Chain, Inc.',
    'Visalia Times Delta':'Gannett Publishing Services dba Visalia Times Delta',
    'GAP':'Gap, Inc.',
    'Gemological Institute of America':'Gemological Institute of America, Inc. (GIA)',
    'Genentech':'Genentech, Inc.',
    'Gibson Overseas':'Gibson Overseas, Inc.',
    'Gilead Sciences':'Gilead Sciences',
    'Gillette Citrus':'Gillette Citrus, Inc.',
    'Ginger.io':'Ginger.io,. Inc.',
    'Global Expedited Transportation':'Global Expedited Transportation, Inc.',
    'Global Medical Response':'Global Medical Response',
    'Google':['Google',
              'Google US-MTV-1900',
              'Google US-MTV-1945',
              'Google US-MTV-1950',
              'Google US-MTV-1965',
              'Google US-MTV-2000',
              'Google US-MTV-40',
              'Google US-MTV-41',
              'Google US-MTV-42',
              'Google US-MTV-43',
              'Google US-MTV-CL2',
              'Google US-MTV-CL3',
              'Google US-MTV-CL4',
              'Google US-MTV-CL5',
              'Google US-MTV-RLS1',
              'Google US-MTV-RLS2',
              'Google US-MTV-STLN2027',
              'Google US-MTV-STLN2029',
              'Google US-MTV-STLN2051',
              'Google US-MTV-STLN2061',
              'Google US-SFO-1MST',
              'Google US-SFO-FRE215',
              'Google US-SFO-MKT1',
              'Google US-SFO-SPE',
              'Google US-SVL-CRSM1225',
              'Google US-SVL-CRSM1260',
              'Google US-SVL-CRSM1265',
              'Google US-SVL-HMBLT1',
              'Google US-SVL-HMBLT3',
              'Google US-SVL-HMBLT4'],
    'Grammarly':'Grammarly, Inc.',
    'Mission Foods':'Gruma Corp dba Mission Foods',
    'HMI':'HMI',
    'HMSHost':['HMSHost',
               'HMSHost located at Admirals Clubs & Lounges at Los Angeles International Airport (LAX)'],
    'Hallcon Corporation':'Hallcon Corporation',
    'Happy Money':'Happy Money, Inc.',
    'Hawker Pacific Aerospace':'Hawker Pacific Aerospace, Inc.',
    'Headspace':'Headspace, Inc.',
    'Headway Technologies':'Headway Technologies Inc.',
    'Henry Mayo Newhall Hospital':'Henry Mayo Newhall Hospital',
    'Herbalife International of America':['Herbalife International of America, Inc',
                                          'Herbalife International of America, Inc.'],
    'Hinge Health, Inc.':'Hinge Health, Inc.',
    'Hollingsworth':'Hollingsworth LLC',
    'Connect Homes':'Home EC, Inc. dba Connect Homes',
    'Hyatt Centric':'Hyatt Centric',
    'Hyperloop': ['Hyperloop One',
                  'Hyperloop Technologies, Inc. dba Hyperloop One'],
    'IKEA':'IKEA Distribution Services, Inc.',
    'ISU PETASYS CORP.':'ISU PETASYS CORP.',
    'ITC Federal':'ITC Federal',
    'Il Fornaio (America)':'Il Fornaio (America) LLC dba Il Fornaio Pasadena Ristorante',
    'Illumina Inc':['Illumina Inc. World Headquarters',
                    'Illumina, Inc.',
                    'Illumina, Inc. Worldwide Headquarters'],
    'Image Solutions Apparel':'Image Solutions Apparel, Inc',
    'ImmunityBio':'ImmunityBio, Inc.',
    'Imperial Bag & Paper Co.':'Imperial Bag & Paper Co. LLC, dba Imperial Dade',
    'In-N-Out Burgers':'In-N-Out Burgers, a California corporation',
    'Individualized ABA Services for Families':'Individualized ABA Services for Families dba IABAS For Families',
    'Informatica':'Informatica LLC',
    'Intel Corporation':'Intel Corporation',
    'Intense Lighting':'Intense Lighting, LLC',
    'Invitae Corporation':'Invitae Corporation',
    'J.B. Hunt Transport':'J.B. Hunt Transport, Inc.',
    'JUUL Labs':['JUUL Labs',
                 'JUUL Labs - 270',
                 'JUUL Labs - 271',
                 'JUUl Labs - 270'],
    'Jabil Inc.':['Jabil Inc.',
                  'Jabil, Inc.'],
    'Jacmar Foodservice Distribution':'Jacmar Foodservice Distribution, LLC',
    'Janssen Research & Development':'Janssen Research & Development, LLC',
    'Jensen Enterprises Inc':'Jensen Enterprises Inc. dba Jensen Precast',
    'John Muir Health (JMH)':'John Muir Health (JMH)',
    'Johns Manville':'Johns Manville',
    'Johnson & Johnson':'Johnson & Johnson Consumer Inc. dba Kenvue',
    'Joy Signal Technology':'Joy Signal Technology',
    'Juniper Networks':'Juniper Networks, Inc.',
    'Juul Labs':['Juul Labs - 270',
                 'Juul Labs, Inc.',
                 'Juul Labs, Inc. - 270'],
    'Kaiser Foundation Hospitals': ['Kaiser Foundation Hospitals',
                                    'Kaiser Foundation Hospitals - Arrow',
                                    'Kaiser Foundation Hospitals - Canyon',
                                    'Kaiser Foundation Hospitals - Fair Oaks',
                                    'Kaiser Foundation Hospitals - Harrison',
                                    'Kaiser Foundation Hospitals - Lakeside',
                                    'Kaiser Foundation Hospitals - Mission',
                                    'Kaiser Foundation Hospitals - One',
                                    'Kaiser Foundation Hospitals - Ordway Building',
                                    'Kaiser Foundation Hospitals - Pasadena',
                                    'Kaiser Foundation Hospitals - Rio',
                                    'Kaiser Foundation Hospitals-Los Robles',
                                    'Kaiser Foundation Hospitals-Walnut'],
    'KeHE Distributors':'KeHE Distributors, Inc.',
    'Kett Engineering Corporation':'Kett Engineering Corporation',
    'Kinecta Federal Credit Union':'Kinecta Federal Credit Union',
    'Kings Garden':'Kings Garden, Inc.',
    'Kitayama Brothers':'Kitayama Brothers Inc.',
    'Knauf Insulation Inc.':'Knauf Insulation Inc.',
    'Knox Attorney Service':'Knox Attorney Service, Inc.',
    'Kraft Heinz Foods Company':'Kraft Heinz Foods Company',
    'Kuubix Global': ['Kuubix Global LLC',
                      'Kuubix Global, LLC'],
    'L3 Technologies':'L3 Technologies, Inc., a wholly owned subsidiary of L3Harris Technologies',
     'LAVO Italian Restaurant':'LAVO Italian Restaurant San Diego',
    'LEER Group':['LEER Group',
                  'LEER Group - 1242'],
    'La Reina, CA Religious Nonprofit Corp':'La Reina, a California religious nonprofit corporation, aka La Reina High School and Middle School',
    'La Tortilla Factory':['La Tortilla Factory',
                           'La Tortilla Factory, Inc.'],
    'Lance Camper Mfg. Corp':'Lance Camper Mfg. Corp',
    'Langmuir Systems':'Langmuir Systems, LLC',
    'Lazy Dog Restaurant and Bar':'Lazy Dog Restaurant and Bar',
    'Legacy Supply Chain Services':'Legacy Supply Chain Services II',
    'Leggett & Platt, Inc.':'Leggett & Platt, Incorporated',
    'LendingClub Bank':'LendingClub Bank',
    'Lendlease Americas Inc.':'Lendlease Americas Inc.',
    'Levi Strauss & Co.':'Levi Strauss & Co.',
    'Levy Premium Foodservice':['Levy Premium Foodservice Limited Partnership',
                                'Levy Premium Foodservice Limited Partnership at The Rose Bowl'],
    'Lifelong Medical Care':'Lifelong Medical Care dba Lifelong Ashby Health Center',
    'Liminex, Inc.':'Liminex, Inc. dba GoGuardian',
    'LinkedIn':'LinkedIn Corporation',
    'Locanabio':'Locanabio, Inc.',
    'Lockheed Martin': ['Lockheed Martin',
                        'Lockheed Martin Space'],
    'Lonza Biologics':'Lonza Biologics, Inc.',
    'Los Angeles Times Communications':'Los Angeles Times Communications LLC',
    'Lumentum Operations':'Lumentum Operations LLC',
    'Lund Motion Products':'Lund Motion Products, Inc.',
    'MV Transportation, Inc.':'MV Transportation, Inc.',
    "Macy's":["Macy's Bayfair Store",
              "Macy's Eagle Rock Plaza Store",
              "Macy's Simi Valley Town Center Store"],
    'Instacart':'Maplebear Inc. dba Instacart',
    'Marriott':'Marriott San Mateo Hotel',
    'Marvell Semiconductor, Inc.':'Marvell Semiconductor, Inc.',
    'MasterBrand Cabinets LLC':'MasterBrand Cabinets LLC',
    'Matheson Inc':['Matheson Flight Extenders, Inc.',
                    'Matheson Postal Services, Inc.',
                    'Matheson Trucking, Inc.'],
    'MaxLite, Inc.':'MaxLite, Inc.',
    'Maxar Space': ['Maxar Space LLC - Charleston',
                    'Maxar Space LLC - Fabian',
                    'Maxar Space LLC - Meadow',
                    'Mazar Space LLC - Leghorn'],
    'MedMen Enterprises':'MedMen Enterprises Inc.',
    'Medtronic':'Medtronic, Inc.',
    "MemorialCare Long Beach Medical Center":"MemorialCare Long Beach Medical Center and Miller Children's and Women's Hospital Long Beach",
    'Mercy Medical Transportation':'Mercy Medical Transportation, Inc.',
    "Javier's Newport Beach":"Mexi-Grill, LLC dba Javier's Newport Beach",
    'Microsoft':'Microsoft',
    "Miyoko's PBC":"Miyoko's PBC",
    'Momentum for Health':'Momentum for Health',
    'Monte Nido & Affiliates':'Monte Nido & Affiliates',
    'The Dog':'Moondoggies, Inc dba The Dog',
    'Morrison Healthcare':'Morrison Healthcare at Adventist Health Bakersfield',
    'NEOTech':'NEOTech',
    'NFI National Distribution Centers':'NFI National Distribution Centers LLC',
    'NRI USA':'NRI USA, LLC',
    'NSI Services':'NSI Services, Inc.',
    'NZXT':'NZXT, Inc.',
    'National Distribution Centers':'National Distribution Centers, LLC',
    'Natural Alternatives International':'Natural Alternatives International',
    'Neovia Logistics Distribution':'Neovia Logistics Distribution, LP',
    'Nestle':'Nestle USA',
    'Nevro Corp.':'Nevro Corp.',
    'Newark Group':'Newark Group, Inc.',
    'Newell Brands Distribution':'Newell Brands Distribution LLC',
    'NextGen Healthcare':'NextGen Healthcare, Inc.',
    'Nextdoor':'Nextdoor, Inc.',
    'Niagara Bottling':'Niagara Bottling LLC',
    'Niantic, Inc.':'Niantic, Inc.',
    'Nobel House Home Furnishings': ['Nobel House Home Furnishings LLC',
                                     'Noble House Home Furnishing LLC Superior HQ',
                                     'Noble House Home Furnishings - 7901 Deering',
                                     'Noble House Home Furnishings LLC',
                                     'Noble House Home Furnishings LLC GDF Outlet',
                                     'Noble House Home Furnishings LLc'],
    'Nordson Corporation':'Nordson Corporation',
    'Nordstrom':'Nordstrom',
    'Northrop Grumman':'Northrop Grumman',
    'NuVasive, Inc.':'NuVasive, Inc.',
    'OTS Solutions':'OTS Solutions, LLC dba On Time Staffing',
    'Ojai Valley Inn':'Ojai Valley Inn',
    'Okta, Inc.':'Okta, Inc.',
    'Oracle America, Inc.':'Oracle America, Inc. at Adventist Health',
    'Organic Milling': ['Organic Milling - Acacia Facility',
                        'Organic Milling - Allen Facility'],
    'Orion Strategic Solutions':'Orion Strategic Solutions LLC',
    'Osso VR, Inc.':'Osso VR, Inc.',
    'Owens-Brockway Glass Container':['Owens-Brockway Glass Container Inc.',
                                      'Owens-Brockway Glass Container, Inc.'],
    'PIRCH':'PIRCH',
    'PKL Services Inc':'PKL Services Inc',
    'PLI Holdings, Inc.':'PLI Holdings, Inc.',
    'PWLC I, Inc.':'PWLC I, Inc.',
    'Pac-12':['Pac-12 Conference',
              'Pac-12 Enterprises, LLC'],
    'Pacific Catch':'Pacific Catch, Inc.',
    'Pacific Coast Sightseeing Tours & Charters':'Pacific Coast Sightseeing Tours & Charters, Inc.',
    'Golden Gate Fields':'Pacific Racing Association dba Golden Gate Fields',
    'Palmdale Regional Medical Center':'Palmdale Regional Medical Center',
    'Para Los Ninos':'Para Los Ninos',
    'Paramount Global':'Paramount Global',
    'Parker Hannifin Corporation':['Parker Hannifin Corporation',
                                   'Parker Hannifin Corporation BWF Division'],
    'Participant Media, LLC':'Participant Media, LLC',
    'PayPal':'PayPal',
    'Peloton Interactive Inc.':'Peloton Interactive Inc.',
    'Pennymac':'Pennymac',
    'Penske Logistics':'Penske Logistics, LLC providing services for Aldi, Inc.',
    'Perfect Day, Inc.':'Perfect Day, Inc.',
    'Performance Plastics, Inc.':'Performance Plastics, Inc.',
    'Personalis':'Personalis, Inc.',
    'PetVet Care Centers':'PetVet Care Centers (California), Inc.',
    'Petco':'Petco Animal Supplies Stores',
    'Pfizer':'Pfizer',
    'PhenomeX Inc.':'PhenomeX Inc.',
    'Phillips Tool & Die, Inc':'Phillips Tool & Die, Inc. dba Busy Bee Tooling',
    'Pixelberry Studios':'Pixelberry Studios',
    'Pizza Antica Mill Valley':'Pizza Antica Mill Valley',
    'Planet Labs PBC':'Planet Labs PBC',
    'Plenty Unlimited Inc.':'Plenty Unlimited Inc.',
    'Point Digital Finance, Inc.':'Point Digital Finance',
    'Post Acute Registry':'Post Acute Registry LLC',
    'Primary Health':'Primary Health',
    'Project Fusion':'Project Fusion, LLC',
    'Prolific Logistics LLC':'Prolific Logistics LLC',
    'Proterra Operating Company':'Proterra Operating Company, Inc.',
    'Providence Holy Cross Medical Center':'Providence Holy Cross Medical Center',
    'Pure Storage':'Pure Storage, Inc.',
    'QG Printing II':'QG Printing II, LLC',
    'Qualcomm Incorporated':'Qualcomm Incorporated',
    'Quality Fabrication Inc.':'Quality Fabrication Inc.',
    'Quanex Homeshield':'Quanex Homeshield LLC',
    'REACH Medical Holdings':'REACH Medical Holdings',
    'RSN Co.':'RSN Co., LLC dba Bally Sports San Diego',
    'Reins International California':'Reins International California, Inc. dba Gyu-Kaku',
    'Renesas Electronics Corporation':'Renesas Electronics Corporation',
    'ResMEd Inc.':'ResMEd Inc.',
    'Resort Vacations, Inc.':'Resort Vacations, Inc.',
    'WIS International':'Retail Services WIS Corporation, dba WIS International',
    "D' Addario Woodwinds": ["Rico Corp. dba D' Addario Woodwinds",
                             "Rico Corp. dba D'Addario Woodwinds"],
    'Ridgecrest Regional Hospital':'Ridgecrest Regional Hospital',
    'Rieke LLC':'Rieke LLC',
    'Riot Games':'Riot Games',
    'South Coast Baking':'Rise Baking Company dba South Coast Baking, LLC',
    'Rivian Automotive':'Rivian Automotive, LLC',
    'Robert Bosch':'Robert Bosch LLC',
    'Robinhood Markets':'Robinhood Markets, Inc.',
    'Rodan & Fields':'Rodan & Fields, LLC',
    'Roku':'Roku, Inc.',
    'Ruan Transport Corporation':'Ruan Transport Corporation',
    'SAP America':'SAP America, Inc.',
    'SCAT Enterprises':'SCAT Enterprises LLC',
    'Solidigm':'SK hynix NAND Product Solutions Corp. dba Solidigm',
    'SMTC Corporation':'SMTC Corporation',
    'SP+ Corporation':'SP+ Corporation at Hollywood Burbank Airport',
    'SPS Ventures':'SPS Ventures Inc.',
    'SPT Microtechnologies USA':'SPT Microtechnologies USA, Inc.',
    'Safeway':'Safeway',
    'Safran Cabin':'Safran Cabin, Inc.',
    'Sakata Seed America':'Sakata Seed America, Inc.',
    'Salesforce':'Salesforce, Inc.',
    'San Diego Imaging Management':'San Diego Imaging Management, Inc.',
    'Sangamo Therapeutics':'Sangamo Therapeutics, Inc.',
    'Sanofi':'Sanofi',
    'Scenario-Cockram USA':'Scenario-Cockram USA, Inc.',
    'Scripps Health':'Scripps Health',
    'Sega of America':'Sega of America, Inc.',
    'Shadow Holdings':['Shadow Holdings, LLC dba Bright Innovation Labs',
                       'Shadow Holdings, LLC.'],
    'Shaw Industries Group':'Shaw Industries Group Inc. Plant WG',
    'Shift Technologies':'Shift Technologies, Inc.',
    'Silgan Containers Manufacturing Corporation':'Silgan Containers Manufacturing Corporation',
    'Snap Inc.':'Snap Inc.',
    'SoCal Pizza Holdings':'SoCal Pizza Holdings, LLC',
    'SoCal South OC':'SoCal South OC LLC',
    'Sober Living by the Sea':'Sober Living by the Sea, Inc.',
    'Sodexo, Inc.':'Sodexo, Inc. and Affiliates at Beverly Hospital',
    'Sonoco':'Sonoco',
    'Sony':'Sony Interactive Entertainment LLC',
    'Sorrento Therapeutics':'Sorrento Therapeutics, Inc.',
    'Southern California Pizza Company':'Southern California Pizza Company LLC',
    'Southern Counties Express':'Southern Counties Express',
    'Spencer Construction':'Spencer Construction, LLC',
    'Splash Cafe':'Splash Cafe, Inc. dba Splash Cafe Artisan Bakery',
    'Splunk Inc.':'Splunk Inc.',
    'Spotify':['Spotify USA Inc',
               'Spotify USA Inc.'],
    'St James Infirmary':'St James Infirmary',
    'St. John of God Health Care Services':'St. John of God Health Care Services',
    'St. Vincent de Paul High School':'St. Vincent de Paul High School',
    'Stanislaus Surgical Hospital':'Stanislaus Surgical Hospital',
    'Steelcase Inc.':'Steelcase Inc.',
    'Stir Foods':'Stir Foods, LLC',
    'Sugar Foods':'Sugar Foods LLC',
    'Sumo Logic':'Sumo Logic, Inc.',
    'Sunrise Growers Inc.':'Sunrise Growers Inc.',
    'Sunrun Inc.': ['Sunrun Inc.',
                    'Sunrun Inc. - Angus'],
    'Surefox North America Inc':'Surefox North America Inc',
    'Surgalign Spine Technologies':'Surgalign Spine Technologies, Inc.',
    'Syngenta Seeds':'Syngenta Seeds, LLC',
    'T. Hasegawa U.S.A.':'T. Hasegawa U.S.A., Inc.',
    'TAO Group Hospitality':'TAO Group Hospitality',
    'TE Connectivity':'TE Connectivity',
    'TIDAL':'TIDAL, a subsidary of Block, Inc.',
    'TMG Hotel':'TMG Hotel Figueroa F&B Venues',
    'TTM Technologies':'TTM Technologies, Inc., subsidiary of TTM Technologies North America, LLC',
    'TVT Trucking, Inc':'TVT Trucking, Inc. and FTU Labor Contractors',
    'Talis Biomedical Corporation':'Talis Biomedical Corporation',
    'Target':'Target Corporation',
    'Tattooed Chef':['Tattooed Chef - 6304',
                     'Tattooed Chef - 6305',
                     'Tattooed Chef - 6309',
                     'Tattooed Chef - 6325',
                     'Tattooed Chef - 6403',
                     'Tattooed Chef - 6409',
                     'Tattooed Chef - 6411',
                     'Tattooed Chef, Inc., wholly owned subsidiary Ittella International, LLC'],
    'Technical Consumer Products':'Technical Consumer Products, Inc.',
    'Telecare Corporation':'Telecare Corporation',
    'Temple Nightclub':'Temple Nightclub',
    'Tempo Automation Holdings':'Tempo Automation Holdings, Inc.',
    'Terre du Soleil':'Terre du Soleil dba Auberge du Soleil',
    'The Apothecarium':'The Apothecarium',
    'The Boeing Company':'The Boeing Company',
    'The Health Trust':'The Health Trust',
    'The Hills Hotel':'The Hills Hotel',
    'The Hotel del Coronado':'The Hotel del Coronado',
    'The Real Good Food Company':'The Real Good Food Company, LLC',
    'The Rinks Foundation':'The Rinks Foundation',
    'The Ronin Project':'The Ronin Project, Inc.',
    'The Vons Companies':'The Vons Companies Inc.',
    'Thermo Fisher Scientific':'Thermo Fisher Scientific',
    'Tower Semiconductor':'Tower Semiconductor',
    'Transdev Services':'Transdev Services, Inc.',
    'Tri-City Medical Center':'Tri-City Medical Center',
    'TriLink Biotechnologies':'TriLink Biotechnologies, LLC and Maravai Intermediate Holdings, LLC',
    'Tropitone Furniture Company':'Tropitone Furniture Company, Inc.',
    'Trove Recommerce':'Trove Recommerce',
    'TuSimple, Inc.':'TuSimple, Inc.',
    'Twitch Interactive, Inc.':'Twitch Interactive, Inc. SFO19 Facility',
    'U.S. Bank': ['U. S. Bank (1296)',
                  'U.S. Bank',
                  'U.S. Bank (1289)',
                  'U.S. Bank (1296)'
                  'U.S. Bank (1296) '],
    'Universal Music Group':'UMG Recordings, Inc. (Universal Music Group)',
    'UPS':'UPS',
    'United Road Services':'URS Midwest, Inc dba United Road Services',
    'US Display Group':'US Display Group',
    'US Motor Works':'US Motor Works, LLC',
    'USS-UPI':'USS-UPI, LLC',
    'Unity Technologies':'Unity Technologies SF',
    'Universal Intermodal Services, Inc.':'Universal Intermodal Services, Inc.',
    'Universal Protection Service':'Universal Protection Service, LP dba Allied Universal Security Services',
    'VF Santa Fe Springs Distribution Center':'VF Santa Fe Springs Distribution Center',
    'Valiant Integrated Services':'Valiant Integrated Services',
    'Valley Processing':'Valley Processing, a HEXPOL Company',
    'Vans HQ':'Vans HQ',
    'Vertical Supply Group':'Vertical Supply Group',
    'Viasat, Inc':['Viasat, Inc',
                   'Viasat, Inc.'],
    'Vionic Consumer Care Operations':'Vionic Consumer Care Operations',
    'Virgin Galactic':'Virgin Galactic, LLC',
    'Visalia Citrus Packing Group':'Visalia Citrus Packing Group, Inc.',
    'Volta Inc.':'Volta Inc.',
    'TCWGlobal':'WMBE Payrolling Inc. dba TCWGlobal',
    'WaFD Bank':'WaFD Bank',
    'Walmart':'Walmart',
    'Walters & Wolf Precast':'Walters & Wolf Precast',
    'Watermark Services':'Watermark Services IV, LLC',
    'Wayfair':'Wayfair, LLC',
    'Lowell Farms':'Wellness Innovation Group Inc. dba Lowell Farms',
    'Wells Fargo':'Wells Fargo',
    'West Coast Industries':'West Coast Industries, Inc.',
    'Western Digital':'Western Digital',
    'Western Power Sports':'Western Power Sports, LLC',
    'William Kreysler & Associates':'William Kreysler & Associates, Inc.',
    'Williams-Sonoma Direct':'Williams-Sonoma Direct, Inc.',
    'Monocle':'Winkontent Limited dba Monocle',
    'Wolfspeed, Inc.':'Wolfspeed, Inc.',
    'Woodbury University':'Woodbury University',
    'Woodland Rite Aid Distribution Center':'Woodland Rite Aid Distribution Center',
    'Workforce Enterprises':'Workforce Enterprises',
    'Xcite Steps':'Xcite Steps, a wholly owned subsidiary of Invo Healthcare, LLC',
    'Xerxes Corporation':'Xerxes Corporation',
    'Yelloh':'Yelloh',
    'Yellow Corporation':['Yellow Corporation',
                          'Yellow Corporation-Batavia St.',
                          'Yellow Corporation-Eckhoff'],
    'Kelly Toys Holding':'Z Global Logistics dba Kelly Toys Holding LLC',
    'Zwift, Inc.':'Zwift, Inc.',
    'Zymergen Inc.':'Zymergen Inc.',
    'dnata US Inflight Catering': ['dnata US Inflight Caterin',
                                   'dnata US Inflight Catering'],
    'eBay Inc.':'eBay Inc.'
}


In [ ]:
#View keys from the dictionary list
print(list(correction_dict.keys()))

#View values from the dictionary list
typo_list = (list(correction_dict.values()))
print(typo_list)

In [ ]:
#This works as a exact match case, cannot match keys with multiple values
list(correction_dict.keys())[list(correction_dict.values()).index('AHMC San Gabriel Valley Medical Center LP dba San Gabriel Valley Medical Center')]

In [ ]:
[i for i in correction_dict if correction_dict[i] == '99 Cents Only Stores']

In [ ]:
#Create a loop that scrolls through the Company column to replay the values with the key in the dict
for company in warn_df['Company']:
    error_list = []
    replaced_list = []

    for key, value in correction_dict.items():
        if value == company:
            correct_word = key
            warn_df['Company'] = warn_df['Company'].replace(company,correct_word)
            replaced_list.append(company)
            print(list(replaced_list))
        else:
            error_list.append(company)
            print(list(error_list))

#dir((warn_df['Company']))


In [ ]:
#Create a loop that scrolls through the Company column to replay the values with the key in the dict
for company in warn_df['Company']:
    
    
    correct_word = list(correction_dict.keys())[list(correction_dict.values()).index(company)]

    if warn_df['Company'] == company:
        warn_df['Company'] = warn_df['Company'].replace(company,correct_word)
    #else 
        
    print(sort(list(warn_df['Company'].unique())))

<h1>Plotting Data</h1>

In [ ]:
#Import Dependencies
import matplotlib.pyplot as plt
import plotly.express as px
#import plotly.offline as pyo
#import plotly.graph_objs as go
import plotly.io as pio

import panel as pn

pn.extension()

# Set notebook mode to work in offline
#pyo.init_notebook_mode()

pio.renderers.default = "notebook" #"vscode"

In [ ]:
#Group data based on county
county_df = warn_df.groupby("County/Parish").describe()

#Save and view columns in form of csv
county_df.to_csv('/Users/leslieleiva/Documents/GitHub/WARN Notice Analysis/Resources/groupby_county.csv')

#Print out column information
print(county_df.info())

In [ ]:
#DF county and number of companies affected by layoffs
county_company_num = warn_df.groupby([warn_df.columns[0],warn_df.columns[5]]).count()

#Delete unnecessary columns by creating a new df to replace the old one
county_company_num = county_company_num[[
    'Notice\nDate'
]]

#Reset index
county_company_num = county_company_num.reset_index()

#Renaming column
county_company_num = county_company_num.rename(columns={
    "Layoff/\nClosure":"Layoff Status",
    county_company_num.columns[2] : 'Number of Companies'
})

#View table
print(county_company_num.head())

#Convert to dictionary for ploting
county_dict = county_company_num.to_dict()
print(county_dict.keys())

In [ ]:
fig = px.sunburst(
    county_dict,
    names='County/Parish',
    parents='Layoff Status',
    values='Number of Companies',
)
fig.show()
#pyo.iplot(fig,filename="plot 1")

<h4> Attempt to read XML files through Python </h4>

In [ ]:
#Import dependencies
import pandas as pd
from io import StringIO
import xml.etree.ElementTree as ET

import xmltodict
from   pathlib import Path

In [ ]:
#Path to XML files
infra_projects = '/Users/leslieleiva/Documents/GitHub/New-York-Construction-Analysis/Resources/NY Active Projects - Infrastructure (Historical).xml'
pub_projects = '/Users/leslieleiva/Documents/GitHub/New-York-Construction-Analysis/Resources/NY Active Projects - Public Buildings (Historical).xml'
capital_projects = '/Users/leslieleiva/Documents/GitHub/New-York-Construction-Analysis/Resources/NY Capital Projects.xml'
housing_data = '/Users/leslieleiva/Documents/GitHub/New-York-Construction-Analysis/Resources/NY Housing Database by 2020 NTA .xml'

#Path to JSON files
capital_schedule_path = '/Users/leslieleiva/Documents/GitHub/New-York-Construction-Analysis/Resources/NY Capital Project Schedules and Budgets.json'

#List of xml paths
xml_path = [
    infra_projects_path,
    pub_projects_path,
    capital_projects_path,
    housing_data_path
]

In [ ]:
pd.read_xml(xml_path[0])

In [ ]:
#Create dictionart list to store df in loop
roots_list = []
xml_dfs = {}

#Read the XMLs and convert to df via loop
for path in xml_path:
    # #Parse xml file
    # tree = ET.parse(path)
    # root = tree.getroot()   
    # print(root)
    # print(root.tag)
    # print(root.attrib)

    # for x in root:
    #     print(x.tag, x.attrib)

    #Read and turn into dataframe
    temp_df = (pd.read_xml(StringIO(path),stylesheet=mentions_xsl)

    #Create a df based on file path
    #xml_dfs["{}_df".format(path)] = temp_df
    print(temp_df)
    df = pd.DataFrame.from_dict(temp_df)
    locals()[df] = df
    print("DataFrame: " + temp_df)

#xml_dfs